![](./banner.png)

# Data Collection 3: UCI Drugs.com reviews dataset

##### Citation
Felix Gräßer, Surya Kallumadi, Hagen Malberg, and Sebastian Zaunseder. 2018. Aspect-Based Sentiment Analysis of Drug Reviews Applying Cross-Domain and Cross-Data Learning. In Proceedings of the 2018 International Conference on Digital Health (DH '18). ACM, New York, NY, USA, 121-125. DOI: [Web Link](https://dl.acm.org/doi/10.1145/3194658.3194677)


## Overview
The dataset includes patient reviews on medicinal products, conditions that the product is indicated for, and a 10-star rating reflecting overall patient satisfaction.
For full description, please refer to [UCI Machine Learning Repositories](https://archive.ics.uci.edu/ml/datasets/Drug+Review+Dataset+%28Drugs.com%29).


Let's first import the necessary packages:

In [2]:
import numpy as np
import pandas as pd

## Load the data
- The original dataset is split into two separate `csvs`, a training set (75%) and a test set (25%). 
- I loaded the datasets using the Pandas function `pd.read_csv()`.
- The files are in `tsv` format (tab-separated values), therefore I loaded the datasets with `delimiter = '\t'`.

In [8]:
# import training set csv
reviews_train = pd.read_csv('/Users/JocelynHo/Desktop/drugs_com_reviews/drugsComTrain_raw.tsv',
                            delimiter = '\t', index_col = 'Unnamed: 0')

# check number of rows, columns
print(reviews_train.shape)

(161297, 6)


In [9]:
# import test set csv
reviews_test = pd.read_csv('/Users/JocelynHo/Desktop/drugs_com_reviews/drugsComTest_raw.tsv',
                            delimiter = '\t', index_col = 'Unnamed: 0')

# check number of rows, columns
print(reviews_test.shape)

(53766, 6)


## Combine training and test datasets
In order to combine this dataset to the Yellow Card dataset I created, I combined them together using `pd.concat()`. Since the original datasets are shuffled (row indices are not in order), I sorted the indices using `.sort_index(inplace = True)`.

In [7]:
# combine both together
reviews = pd.concat([reviews_test, reviews_train])

# check number of rows and columns
print(reviews.shape)

# resort indices to be in order
reviews.sort_index(inplace = True)

# check first 5 rows
reviews.head()

(215063, 6)


,drugName,condition,review,rating,date,usefulCount
0,Medroxyprogesterone,Abnormal Uterine Bleeding,"""Been on the depo injection since January 2015...",3.0,"October 28, 2015",4
2,Medroxyprogesterone,Amenorrhea,"""I&#039;m 21 years old and recently found out ...",10.0,"October 27, 2015",11
3,Medroxyprogesterone,Abnormal Uterine Bleeding,"""I have been on the shot 11 years and until a ...",8.0,"October 27, 2015",7
4,Medroxyprogesterone,Birth Control,"""Ive had four shots at this point. I was on bi...",9.0,"October 26, 2015",12
5,Medroxyprogesterone,Abnormal Uterine Bleeding,"""I had a total of 3 shots. I got my first one ...",1.0,"October 25, 2015",4


Let's take a look at the columns:
1. `drugName` (categorical): Brand name of drug
2. `condition` (categorical): Name of condition drug is used for
3. `review` (text): Patient's review
4. `rating` (numerical): Patient's rating out of 10 stars
5. `date` (date): Date of review entry
6. `usefulCount` (numerical): Number of users who found review useful

## Data Cleaning
### Drop date column
`date` column is not needed, so I decided to drop it.

In [11]:
# drop date column
reviews.drop(columns = 'date', inplace = True)

### Rename columns
I didn't like how the column names were set up, so I decided to rename them.

In [12]:
# rename columns
reviews.columns = ['drug', 'condition', 'review', 'rating', 'useful_count']

### Convert all text to lower case
I created a custom function `to_lower()` that converts all strings of texts to lower case, for easier manipulation at later stages. The try-except clause makes sure that if certain values cannot be converted (e.g. nan values), instead of raising an error, the original value will be returned.

In [13]:
# create function to convert to lowercase
def to_lower(x):
    try:
        return x.lower()
    except:
        return x

# apply custom function to the entire dataframe
reviews = reviews.applymap(to_lower)

### Convert brand names to drug names
The biggest task of this dataset is to address the issue of brand names vs drug names. This dataset is based on each medicinal product's BRAND NAME (e.g. Calpol), instead of the DRUG NAME or active ingredient (e.g. paracetamol).

In order to combine this dataset to my Yellow Card dataset, I have to convert all the brand names to their respective drug names.

##### Trial - manual conversion
Initially I tried to manually convert the brand names to drug names, but quickly realised it is too time consuming and decided to then automate the entire process.

Here is a snippet of my attempt to manually convert the values

In [533]:
# change brand names to drug names
reviews.loc[reviews[reviews['drug'] == 'acanya'].index, 'drug'] = 'benzoyl peroxide / clindamycin'
reviews.loc[reviews[reviews['drug'] == 'abilify discmelt'].index, 'drug'] = 'aripiprazole'
reviews.loc[reviews[reviews['drug'] == 'acnex'].index, 'drug'] = 'salicyic acid'

In [ ]:
{'acanya': 'benzoyl peroxide / clindamycin',
'abilify discmelt', 'aripiprazole',
'acnex': 'salicyic acid',
'a / b otic': 'antipyrine / benzocaine',
'a + d cracked skin relief': 'benzalkonium chloride / lidocaine',
'actifed': 'chlorphenamine / phenylephrine',
'activella': 'estradiol / norethindrone',
'actoplus met': 'metformin / pioglitazone',
'actonel with calcium': 'risedronate / calcium',
'adderall xr': 'amphetamine',
'addyi': 'flibanserin',
'advicor': 'niacin / lovastatin',
'advil cold and sinus': 'ibuprofen / pseudoephedrine',
'advil liqui-gels': 'ibuprofen / pseudoephedrine',
'advil migraine': 'ibuprofen',
'afrin original': 'oxymetazoline',
'afrin sinus': 'oxymetazoline',
'ala-quin': 'clioquinol / hydrocortisone',
'alahist lq': 'diphenhydramine / phenylephrine',
'alavert': 'loratidine',
'aleve-d sinus & cold': 'naproxen / pseudoephedrine',
'alka-seltzer plus cold formula sparkling original effervescent tablets': 'aspirin / chlorpheniramine / phenylephrine'}

#### Extract unique brand names
In order to automate the process, I first extracted all the unique brand names using `.unique()` on the `drug` column.

In [105]:
brands = list(reviews.drug.unique())
print(len(brands))
brands[:20]

3671


['medroxyprogesterone',
 'phenylephrine',
 'silodosin',
 'resorcinol / sulfur',
 'methylin er',
 'clemastine',
 'pentosan polysulfate sodium',
 'metoprolol tartrate',
 'xeljanz xr',
 'rituxan',
 'torsemide',
 'diphenhydramine / hydrocortisone',
 'levo-dromoran',
 'everolimus',
 'tolak',
 'fenoprofen',
 'metronidazole',
 'sumatriptan',
 'ibudone',
 'triple antibiotic']

#### Clean names to use in Drugs.com
First, I need to change the format of the brand names, so that they follow the correct format of Drugs.com urls. This is done using a list comprehension with the following:
- Replacing `'/'` with `'-'`
- Replacing `' '` (space) with `'-'`
- Replacing `' #'` with `'-'`
- Replacing `"'"` with `'-'`
- Replacing `' + '` with `'-'`
- Removing extra spaces

In [108]:
# create dictionary containing symbols to be removed
x = {' / ': '-', ' #': '-', "'": "-", ' ': '-', ' + ': '-', '+': '-'}
# create empty lsit to store new brand names
brands2 = []

# loop through each brand name and remove symbols
for i in brands:
    for a,b in x.items():
        i = i.replace(a, b)
    brands2.append(i)

# check length
print(len(brands2))
# show top 20 values
brands2[:20]

3671


['medroxyprogesterone',
 'phenylephrine',
 'silodosin',
 'resorcinol-sulfur',
 'methylin-er',
 'clemastine',
 'pentosan-polysulfate-sodium',
 'metoprolol-tartrate',
 'xeljanz-xr',
 'rituxan',
 'torsemide',
 'diphenhydramine-hydrocortisone',
 'levo-dromoran',
 'everolimus',
 'tolak',
 'fenoprofen',
 'metronidazole',
 'sumatriptan',
 'ibudone',
 'triple-antibiotic']

#### Retrieve active ingredients of products using BeautifulSoup
##### Import libraries
BeautifulSoup lets users extract useful data from html. I first imported the libraries:

In [25]:
import requests
import bs4
from bs4 import BeautifulSoup
from time import sleep
import random
import string
from tqdm import tqdm

##### Extract drug names from Drugs.com
Since reviews from this dataset are extracted from Drugs.com, I decided to extract the respective active ingredients (drug names) from the same site. Work flow will be as follows:
- I first created an empty lists `actives`, this will store all the data I extract from Drugs.com.
- Then, for each brand in the `brands` list I created above, I try to access their Drugs.com pages by placing each brand name into the url.
- If the webpage is accessible, I then extract the `'Generic Name'` listed on the page.
- If not, I then try different web pages (e.g. drug interactions page or side effects page) to extract the same information.
- If that doesn't work, I simply append a `nan` value to the `actives` list, indicating that the information was not retrievable using this particular method.

In [ ]:
# create empty list to store active ingredients
actives = []

# loop through each brand name
for i in tqdm(brands2):
    
    # wait for a random number of seconds, from 3 to 7 seconds
    sleep(random.randint(3, 7))
    
    try:
        
        # insert each brand name into the url
        url = f'https://www.drugs.com/mtm/{i}.html'
        # get a request using the url
        r = requests.get(url)
        # find the necessary information using beautifulsoup
        soup = BeautifulSoup(r.text, 'html.parser')
        results = soup.find("div", class_ = 'contentBox')
        names = results.find_next("p").text


        # if webpage exists, text starts with 'Generic name:'
        if names.startswith('Generic'):

            # some texts have a long list, we only need the first part before '\n'
            try:
                try:
                    generic = names.split(' (')[0].split(': ')[1].split('\n')[0]
                    actives.append(generic)
                except:
                    generic = names.split(' (')[0].split(': ')[1]
                    actives.append(generic)
            except:
                actives.append(np.nan)

        # if the /mtm/ url does not work, try with side-effects.html
        elif names.startswith('Sorry'):
            try:
                # insert each brand name into the url
                url = f'https://www.drugs.com/sfx/{i}-side-effects.html'
                # get a request using the url
                r = requests.get(url)
                # find the necessary information using beautifulsoup
                soup = BeautifulSoup(r.text, 'html.parser')
                results = soup.find("div", class_ = 'contentBox')
                names = results.find_next("p").text

                # only extract the generic names after 'Generic Name: '
                try:
                    actives.append(names.split(': ')[1])

                # safety net to catch any errors and return nan value
                except:
                    actives.append(np.nan)

            # if webpage not accesible, return nan value
            except:
                actives.append(np.nan)

        else:
            actives.append(np.nan)
            
    except:
        actives.append(np.nan)

  2%|▏         | 59/3671 [05:49<5:58:13,  5.95s/it]

In [117]:
print(len(actives))
actives[:5]

4485


['medroxyprogesterone', 'phenylephrine', 'silodosin', nan, nan]

In [ ]:
            try:
                # insert each brand name into the url
                url = f'https://www.drugs.com/pro/{i}.html'
                # get a request using the url
                r = requests.get(url)
                # find the necessary information using beautifulsoup
                soup = BeautifulSoup(r.text, 'html.parser')
                results = soup.find("div", class_ = 'contentBox')
                names = results.find_next("p").text

                # if webpage exists, text starts with 'Generic name:'
                if names.startswith('Generic'):

                    # some texts have a long list, we only need the first part before '\n'
                    try:
                        try:
                            generic = names.split(' (')[0].split(': ')[1].split('\n')[0]
                            actives.append(generic)
                        except:
                            generic = names.split(' (')[0].split(': ')[1]
                            actives.append(generic)
                    except:
                        actives.append(np.nan)
            # if webpage not accesible, return nan value
            except:

##### Create dictionary of brand names to drug names
After retrieving the active ingredients (drug names) of each brand, I then created a dictionary that pairs each brand name to the drug names.

`zip` pairs each element of the same index in both the `brands` and `actives` list together, and returns a tuple of (brand name, drug names). `i[0]` extracts the brand name, `i[1]` extracts the drug names. I then used the dictionary comprehension method to create the desired dictionary.

In [ ]:
brand_drug_dict = {i[0]:i[1] for i in zip(brands, actives)}

##### Check for null values
I checked for the brand names that I was unable to scrape from Drugs.com:

In [ ]:
not_found = [i for i, j in brand_drug_dict.items() if str(j) == 'nan']
print(len(not_found))
not_found

In [534]:
reviews[reviews['drug'].str.contains(r' / \d')]

,drug,condition,review,rating,useful_count
3604,lorcet 10 / 650,back pain,"""this medication gave me complete and total re...",10.0,19
3605,lorcet 10 / 650,back pain,"""very effective, helps daily motivation in ter...",9.0,32
3606,lorcet 10 / 650,back pain,"""i have arthritis in my back, and without this...",10.0,23
3607,lorcet 10 / 650,back pain,"""i have a back injury of my l1 through to l5 v...",9.0,33
3608,lorcet 10 / 650,back pain,"""i have been taking lorcet for about eight yrs...",9.0,31
...,...,...,...,...,...
228794,loestrin fe 1 / 20,birth control,"""i was on this for years for birth control and...",9.0,1
228795,loestrin fe 1 / 20,birth control,"""i had been on beyaz for 4 years before this p...",7.0,2
228796,loestrin fe 1 / 20,acne,"""this pill is probably good for a lot of peopl...",2.0,1
228797,loestrin fe 1 / 20,polycystic ovary syndrome,"""this pill has given me nothing but trouble. i...",1.0,1


In [535]:
# separate names with multiple ingredients
combos = reviews[reviews['drug'].str.contains(r'\D\D\D+ / \D\D\D+')]
print(len(combos))
combos.head()

24993


,drug,condition,review,rating,useful_count
868,resorcinol / sulfur,acne,"""have loved this product since i began using i...",10.0,3
992,diphenhydramine / hydrocortisone,pruritus,"""hc derma-pax was my &quot;go to&quot; anti-it...",10.0,0
993,diphenhydramine / hydrocortisone,pruritus,"""i am an avid hiker and for bug bites, poison ...",10.0,4
1723,desloratadine / pseudoephedrine,allergic rhinitis,"""i have alot of allergy issues being in texas,...",9.0,1
1877,mestranol / norethindrone,birth control,"""i&#039;ve been on necon for about a week now....",5.0,0


In [536]:
combos['drug'].unique()

array(['resorcinol / sulfur', 'diphenhydramine / hydrocortisone',
       'desloratadine / pseudoephedrine', 'mestranol / norethindrone',
       'acetaminophen / dextromethorphan / doxylamine / pseudoephedrine',
       'acetaminophen / pamabrom', 'acrivastine / pseudoephedrine',
       'dextromethorphan / quinidine', 'piperonyl butoxide / pyrethrins',
       'atropine / diphenoxylate',
       'acetaminophen / aspirin / caffeine / salicylamide',
       'chlorphenamine / phenylephrine',
       'acetaminophen / dextromethorphan / pseudoephedrine',
       'nystatin / triamcinolone', 'piperacillin / tazobactam',
       'estradiol / norethindrone',
       'acetaminophen / pamabrom / pyrilamine', 'formoterol / mometasone',
       'sodium biphosphate / sodium phosphate',
       'bacitracin / neomycin / polymyxin b',
       'ibuprofen / pseudoephedrine',
       'drospirenone / ethinyl estradiol / levomefolate calcium',
       'caffeine / ergotamine', 'benzoyl peroxide / clindamycin',
       'elb

In [537]:
non_combos = reviews.copy()
non_combos.drop(combos.index, inplace = True)
print(len(non_combos))
non_combos.head()

190070


,drug,condition,review,rating,useful_count
0,medroxyprogesterone,abnormal uterine bleeding,"""been on the depo injection since january 2015...",3.0,4
2,medroxyprogesterone,amenorrhea,"""i&#039;m 21 years old and recently found out ...",10.0,11
3,medroxyprogesterone,abnormal uterine bleeding,"""i have been on the shot 11 years and until a ...",8.0,7
4,medroxyprogesterone,birth control,"""ive had four shots at this point. i was on bi...",9.0,12
5,medroxyprogesterone,abnormal uterine bleeding,"""i had a total of 3 shots. i got my first one ...",1.0,4


In [538]:
# check separated correctly
len(non_combos) + len(combos) == len(reviews)

True

In [539]:
# replace numbers with nothing
combos['drug'] = combos['drug'].str.replace('70 / 30', '')
combos['drug'] = combos['drug'].str.replace(' 3350', '')

# function to separate drug name into list of different drugs
def separate_name(x):
        return x['drug'].split(' / ')

# turn drugName column into a list
combos['drug'] = combos.apply(separate_name, axis = 1)

# duplicate row so that each row has one drug frmo list of drugNames
combos = combos.apply(lambda x: x.explode())

print(len(combos))
combos.head()

<ipython-input-539-d5e4aa8817dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combos['drug'] = combos['drug'].str.replace('70 / 30', '')
<ipython-input-539-d5e4aa8817dc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combos['drug'] = combos['drug'].str.replace(' 3350', '')


52715


<ipython-input-539-d5e4aa8817dc>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combos['drug'] = combos.apply(separate_name, axis = 1)


,drug,condition,review,rating,useful_count
868,resorcinol,acne,"""have loved this product since i began using i...",10.0,3
868,sulfur,acne,"""have loved this product since i began using i...",10.0,3
992,diphenhydramine,pruritus,"""hc derma-pax was my &quot;go to&quot; anti-it...",10.0,0
992,hydrocortisone,pruritus,"""hc derma-pax was my &quot;go to&quot; anti-it...",10.0,0
993,diphenhydramine,pruritus,"""i am an avid hiker and for bug bites, poison ...",10.0,4


In [540]:
combos['drug'].unique()

array(['resorcinol', 'sulfur', 'diphenhydramine', 'hydrocortisone',
       'desloratadine', 'pseudoephedrine', 'mestranol', 'norethindrone',
       'acetaminophen', 'dextromethorphan', 'doxylamine', 'pamabrom',
       'acrivastine', 'quinidine', 'piperonyl butoxide', 'pyrethrins',
       'atropine', 'diphenoxylate', 'aspirin', 'caffeine', 'salicylamide',
       'chlorphenamine', 'phenylephrine', 'nystatin', 'triamcinolone',
       'piperacillin', 'tazobactam', 'estradiol', 'pyrilamine',
       'formoterol', 'mometasone', 'sodium biphosphate',
       'sodium phosphate', 'bacitracin', 'neomycin', 'polymyxin b',
       'ibuprofen', 'drospirenone', 'ethinyl estradiol',
       'levomefolate calcium', 'ergotamine', 'benzoyl peroxide',
       'clindamycin', 'elbasvir', 'grazoprevir', 'amphetamine',
       'dextroamphetamine', 'buprenorphine', 'naloxone', 'coal tar',
       'salicylic acid', 'naphazoline', 'pheniramine', 'etonogestrel',
       'guaifenesin', 'hydrocodone', 'hydrochlorothiazide

In [1672]:
sulfur = combos[combos.drug == 'sulfur'].copy()

In [541]:
# combine new combos and non_combos back together
rv_exploded = pd.concat([combos, non_combos], axis = 0)
print(len(rv_exploded))
rv_exploded.head()

242785


,drug,condition,review,rating,useful_count
868,resorcinol,acne,"""have loved this product since i began using i...",10.0,3
868,sulfur,acne,"""have loved this product since i began using i...",10.0,3
992,diphenhydramine,pruritus,"""hc derma-pax was my &quot;go to&quot; anti-it...",10.0,0
992,hydrocortisone,pruritus,"""hc derma-pax was my &quot;go to&quot; anti-it...",10.0,0
993,diphenhydramine,pruritus,"""i am an avid hiker and for bug bites, poison ...",10.0,4


In [543]:
brands = list(rv_exploded.groupby('drug').count().index)
brands

['abacavir',
 'abatacept',
 'abilify',
 'abilify maintena',
 'abiraterone',
 'abobotulinumtoxina',
 'abraxane',
 'abreva',
 'absorbine jr.',
 'absorica',
 'abstral',
 'acamprosate',
 'acarbose',
 'accolate',
 'accupril',
 'accutane',
 'acebutolol',
 'acetaminophen',
 'acetazolamide',
 'acetic acid',
 'acetohydroxamic acid',
 'acetylcysteine',
 'acidophilus',
 'aciphex',
 'acitretin',
 'aclidinium',
 'acrivastine',
 'actemra',
 'acthar',
 'acticlate',
 'actiq',
 'actonel',
 'actos',
 'acular ls',
 'acuvail',
 'acyclovir',
 'aczone',
 'adalat cc',
 'adalimumab',
 'adapalene',
 'adcirca',
 'adderall',
 'adempas',
 'adenosine',
 'adipex-p',
 'adipost',
 'adoxa',
 'adriamycin',
 'advair diskus',
 'advair hfa',
 'advate',
 'advil',
 'advil cold and sinus liqui-gels',
 'advil pm',
 'adzenys xr-odt',
 'aerobid',
 'aerobid-m',
 'afatinib',
 'afinitor',
 'aflibercept',
 'afrezza',
 'afrin',
 'afrin 4 hour extra moisturizing',
 'afrin nodrip sinus',
 'aftera',
 'aggrenox',
 'airduo respiclick',
 

In [513]:
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
from time import sleep
import random
import pandas as pd
from tqdm import tqdm
import string


In [544]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# PATH = "/Applications/chromedriver"
# driver = webdriver.Chrome(PATH)

options = webdriver.ChromeOptions()
chrome_options=options
options.add_argument('--enable-javascript')
driver = webdriver.Chrome(executable_path='/Applications/chromedriver', options=options)

driver.get('https://go.drugbank.com/drugs/DB00316')

In [545]:
db_ids = []

for x in tqdm(brands):
    try:
        # click on search box
        driver.find_element_by_xpath('/html/body/header/nav[2]/div[1]/form/div[2]').click()
        # enter search keyword
        searchbox = driver.find_element_by_xpath('//*[@id="query"]')
        # clear search box
        searchbox.clear()
        # enter search word
        searchbox.send_keys(x)
        # click enter
        searchbox.send_keys(Keys.ENTER)
        
        # wait 5 seconds
        sleep(5)
        
        try:
            # get DrugBank number and add to dataframe
            num = driver.find_element_by_xpath('/html/body/main/div/div/div[2]/div[2]/dl[1]/dd[2]').text
            db_ids.append(num)
            print('a', x, num)
        
        except:
            # get DrugBank number using class
            num = driver.find_element_by_class_name("col-xl-4 col-md-9 col-sm-8").text
            db_ids.append(num)
            print('b', x, num)
            
        # sleep
        sleep(random.randint(5, 8))
    
    except:
        print('c')
        db_ids.append(np.nan)

  0%|          | 0/3455 [00:00<?, ?it/s]

a abacavir DB01048


  0%|          | 1/3455 [00:12<12:04:50, 12.59s/it]

a abatacept DB01281


  0%|          | 2/3455 [00:24<12:00:18, 12.52s/it]

...

100%|█████████▉| 3453/3455 [10:38:30<00:21, 10.70s/it]

a zyvox DB00601


100%|█████████▉| 3454/3455 [10:38:44<00:11, 11.76s/it]

a zzzquil DB01075


100%|██████████| 3455/3455 [10:38:56<00:00, 11.10s/it]


In [935]:
a = brand_ids.copy()
a.update(dict_x)
a

{'abacavir': 'DB01048',
 'abatacept': 'DB01281',
 'abilify': 'DB01238',
 'abilify maintena': 'DB01238',
 'abiraterone': 'DB05812',
 'abobotulinumtoxina': 'DB00083',
 'abraxane': 'DB01229',
 'abreva': 'DB00632',
 'absorbine jr.': 'DB00825',
 'absorica': 'DB00982',
 'abstral': 'DB00813',
 'acamprosate': 'DB00659',
 'acarbose': 'DB00284',
 'accolate': 'DB00549',
 'accupril': 'DB00881',
 'accutane': 'DB00982',
 'acebutolol': 'DB01193',
 'acetaminophen': 'DB00316',
 'acetazolamide': 'DB00819',
 'acetic acid': 'DB03166',
 'acetohydroxamic acid': 'DB00551',
 'acetylcysteine': 'DB06151',
 'acidophilus': 'DB15823',
 'aciphex': 'DB01129',
 'acitretin': 'DB00459',
 'aclidinium': 'DB08897',
 'acrivastine': 'DB09488',
 'actemra': 'DB06273',
 'acthar': 'DB01285',
 'acticlate': 'DB00254',
 'actiq': 'DB00813',
 'actonel': 'DB00884',
 'actos': 'DB01132',
 'acular ls': 'DB00465',
 'acuvail': 'DB00465',
 'acyclovir': 'DB00787',
 'aczone': 'DB00250',
 'adalat cc': 'DB01115',
 'adalimumab': 'DB00051',
 'ad

In [1207]:
a['senna'] = 'DB11365'

In [970]:
# ids = pd.DataFrame({'brand': brand_ids.keys(), 'db_id': brand_ids.values()})
ids = pd.DataFrame({'brand': a.keys(), 'db_id': a.values()})
ids.tail(10)

,brand,db_id
3445,zymine,[DB00809]
3446,zyprexa,DB00334
3447,zyprexa intramuscular,DB00334
3448,zyprexa zydis,DB00334
3449,zyrtec,DB00341
3450,zyrtec-d,[DB00788]
3451,zyrtec-d 12 hour,[DB13867]
3452,zytiga,DB05812
3453,zyvox,DB00601
3454,zzzquil,DB01075


In [975]:
len(ids.loc[3451].db_id), len(ids.loc[3452].db_id)

(1, 7)

In [980]:
def extract_list(x):
    try:
        if len(x) == 1:
            return x[0]
        else:
            return x
    except:
        return x
ids.db_id = ids.db_id.apply(extract_list)
ids

,brand,db_id
0,abacavir,DB01048
1,abatacept,DB01281
2,abilify,DB01238
3,abilify maintena,DB01238
4,abiraterone,DB05812
...,...,...
3450,zyrtec-d,DB00788
3451,zyrtec-d 12 hour,DB13867
3452,zytiga,DB05812
3453,zyvox,DB00601


In [998]:
scrape_1 = list(ids[ids.db_id.isnull()].brand)
print(len(scrape_1))
scrape_2 = [x.replace(' ', '-') for x in scrape_1]

123


In [993]:
from tqdm import tqdm

actives = []

for i in tqdm(scrape_2):
    try:
        sleep(random.randint(3, 7))
        url = f'https://www.drugs.com/mtm/{i}.html'
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        results = soup.find("div", class_ = 'contentBox')
        names = results.find_next("p").text
        
        if names.startswith('Sorry'):
            actives.append(np.nan)
        elif names.startswith('Generic'):
            actives.append(names)
        else:
            actives.append(names)
                
    except:
        actives.append(np.nan)

100%|██████████| 123/123 [10:43<00:00,  5.23s/it]


In [999]:
ids['actives'] = ids.brand.map({i[0]:i[1] for i in zip(scrape_1, actives)})

In [1168]:
tmp_1 = ids[ids.db_id.isnull()].copy()
tmp_1.actives = tmp_1.actives.str.replace('Generic Name: ', '')
tmp_1.actives = tmp_1.actives.str.extract(r'(^\D+)\(\D+')
tmp_1

,brand,db_id,actives
49,advair hfa,NaN,fluticasone and salmeterol
121,altavera,NaN,ethinyl estradiol and levonorgestrel
170,anoro ellipta,NaN,umeclidinium and vilanterol
193,aquoral,NaN,saliva substitutes
246,augmentin es-600,NaN,amoxicillin and clavulanate potassium
...,...,...,...
3338,xenaderm,NaN,"balsam Peru, castor oil, and trypsin topical"
3355,xylocaine jelly,NaN,lidocaine topical
3422,zomig-zmt,NaN,zolmitriptan (oral)
3430,zosyn,NaN,piperacillin and tazobactam


In [1169]:
tmp_1.actives = tmp_1.actives.str.replace('(', '')
tmp_1.actives = tmp_1.actives.str.replace(')', '')
tmp_1.actives = tmp_1.actives.str.replace(' oral', '')
tmp_1.actives = tmp_1.actives.str.replace(' nasal', '')
tmp_1.actives = tmp_1.actives.str.replace(' topical', '')
tmp_1.actives = tmp_1.actives.str.replace(' vaginal', '')
tmp_1.actives = tmp_1.actives.str.replace(' ophthalmic', '')
tmp_1.actives = tmp_1.actives.str.replace(' inhalation', '')
tmp_1.actives = tmp_1.actives.str.replace('/injection', '')
tmp_1.actives = tmp_1.actives.str.replace(' injection', '')
tmp_1.actives = tmp_1.actives.str.replace(' transdermal skin patch', '')
tmp_1.actives = tmp_1.actives.str.replace(' B BAS i TRAY sin, NEE oh MYE sin, POL ee MIX in B ', '')
tmp_1.actives = tmp_1.actives.str.replace(' and trypsin', ' trypsin')
tmp_1

,brand,db_id,actives
49,advair hfa,NaN,fluticasone and salmeterol
121,altavera,NaN,ethinyl estradiol and levonorgestrel
170,anoro ellipta,NaN,umeclidinium and vilanterol
193,aquoral,NaN,saliva substitutes
246,augmentin es-600,NaN,amoxicillin and clavulanate potassium
...,...,...,...
3338,xenaderm,NaN,"balsam Peru, castor oil, trypsin"
3355,xylocaine jelly,NaN,lidocaine
3422,zomig-zmt,NaN,zolmitriptan
3430,zosyn,NaN,piperacillin and tazobactam


In [1170]:
# replace all commas with and to split in one go
tmp_1.actives = tmp_1.actives.str.replace(', ', ' and ')
tmp_1.actives = tmp_1.actives.str.replace(', and ', ' and ')

def split_and(x):
    try:
        return [i.strip() for i in x.split(' and ')]
    except:
        return x
    
tmp_1.actives = tmp_1.actives.apply(split_and)


In [1208]:
# tmp_1_explode = tmp_1.apply(lambda x: x.explode())
# tmp_1_explode.actives = tmp_1_explode.actives.str.replace('and ', '')
tmp_1_explode.db_id = tmp_1_explode.actives.map(a)
tmp_1_explode

,brand,db_id,actives
49,advair hfa,DB13867,fluticasone
49,advair hfa,DB00938,salmeterol
121,altavera,DB00977,ethinyl estradiol
121,altavera,DB00367,levonorgestrel
170,anoro ellipta,DB09076,umeclidinium
...,...,...,...
3422,zomig-zmt,DB00315,zolmitriptan
3430,zosyn,DB00319,piperacillin
3430,zosyn,DB01606,tazobactam
3442,zylet,DB00873,loteprednol


In [1209]:
tmp_1_explode.isnull().sum()

brand       0
db_id      43
actives    40
dtype: int64

In [1210]:
tmp_1_explode[tmp_1_explode.db_id.isnull()]

,brand,db_id,actives
384,blistex,NaN,topical emollients
385,blistex lip balm,NaN,topical emollients
532,cepacol dual relief sore throat,NaN,NaN
686,coricidin hbp cold & flu,NaN,NaN
713,cryselle,NaN,NaN
801,derma-smoothe,NaN,NaN
925,drixoral cold,NaN,NaN
1030,epiceram,NaN,NaN
1033,epiduo forte,NaN,NaN
1206,flu,NaN,NaN


In [1212]:
tmp_1_explode.dropna(inplace = True)

In [1214]:
tmp_1 = tmp_1_explode.groupby('brand').agg(pd.Series.tolist)
tmp_1

,db_id,actives
brand,,
advair hfa,"[DB13867, DB00938]","[fluticasone, salmeterol]"
altavera,"[DB00977, DB00367]","[ethinyl estradiol, levonorgestrel]"
anoro ellipta,"[DB09076, DB09082]","[umeclidinium, vilanterol]"
aquoral,[[DB09241]],[saliva substitutes]
augmentin es-600,"[DB01060, DB00766]","[amoxicillin, clavulanate potassium]"
...,...,...
xenaderm,"[DB11113, DB11237]","[castor oil, trypsin]"
xylocaine jelly,[DB00281],[lidocaine]
zomig-zmt,[DB00315],[zolmitriptan]


In [1218]:
tmp_1_dict = {i[0]:i[1] for i in zip(tmp_1.index, tmp_1.db_id)}
tmp_1_dict

{'advair hfa': ['DB13867', 'DB00938'],
 'altavera': ['DB00977', 'DB00367'],
 'anoro ellipta': ['DB09076', 'DB09082'],
 'aquoral': [['DB09241']],
 'augmentin es-600': ['DB01060', 'DB00766'],
 'avandamet': ['DB00331', 'DB00412'],
 'ayr saline nasal': ['DB09153'],
 'capmist dm': ['DB00514', 'DB00874', 'DB00852'],
 'capzasin-hp': ['DB06774'],
 'chondroitin': ['DB09301'],
 'claritin-d': ['DB00455', 'DB00852'],
 'claritin-d 12 hour': ['DB00455', 'DB00852'],
 'claritin-d 24 hour': ['DB00455', 'DB00852'],
 'cosopt': ['DB00869', 'DB00373'],
 'creon': ['DB00085'],
 'darvocet a500': ['DB00316', 'DB00647'],
 'darvocet-n 50': ['DB00316', 'DB00647'],
 'diovan hct': ['DB00999', 'DB00177'],
 'donnatal': ['DB13913', 'DB01174'],
 'drixoral cold and allergy': ['DB00405', 'DB00852'],
 'drixoral decongestant non-drowsy': ['DB00852'],
 'ed a-hist': ['DB01114', 'DB00388'],
 'emend': ['DB00673'],
 'emla': ['DB00281', 'DB00750'],
 'emoquette': ['DB00977', 'DB00304'],
 'enstilar': ['DB00443', 'DB02300'],
 'estr

In [1235]:
tmp_1_dict_actives = {i[0]:i[1] for i in zip(tmp_1.index, tmp_1.actives)}
tmp_1_dict_actives

{'advair hfa': ['fluticasone', 'salmeterol'],
 'altavera': ['ethinyl estradiol', 'levonorgestrel'],
 'anoro ellipta': ['umeclidinium', 'vilanterol'],
 'aquoral': ['saliva substitutes'],
 'augmentin es-600': ['amoxicillin', 'clavulanate potassium'],
 'avandamet': ['metformin', 'rosiglitazone'],
 'ayr saline nasal': ['sodium chloride'],
 'capmist dm': ['dextromethorphan', 'guaifenesin', 'pseudoephedrine'],
 'capzasin-hp': ['capsaicin'],
 'chondroitin': ['chondroitin sulfate'],
 'claritin-d': ['loratadine', 'pseudoephedrine'],
 'claritin-d 12 hour': ['loratadine', 'pseudoephedrine'],
 'claritin-d 24 hour': ['loratadine', 'pseudoephedrine'],
 'cosopt': ['dorzolamide', 'timolol'],
 'creon': ['pancrelipase'],
 'darvocet a500': ['acetaminophen', 'propoxyphene'],
 'darvocet-n 50': ['acetaminophen', 'propoxyphene'],
 'diovan hct': ['hydrochlorothiazide', 'valsartan'],
 'donnatal': ['belladonna', 'phenobarbital'],
 'drixoral cold and allergy': ['dexbrompheniramine', 'pseudoephedrine'],
 'drixora

In [1227]:
# copy tmp_1 to ids
x = list(ids[ids.db_id.isnull()].brand.map(tmp_1_dict).values)
x

[['DB13867', 'DB00938'],
 ['DB00977', 'DB00367'],
 ['DB09076', 'DB09082'],
 [['DB09241']],
 ['DB01060', 'DB00766'],
 ['DB00331', 'DB00412'],
 ['DB09153'],
 nan,
 nan,
 ['DB00514', 'DB00874', 'DB00852'],
 ['DB06774'],
 nan,
 ['DB09301'],
 ['DB00455', 'DB00852'],
 ['DB00455', 'DB00852'],
 ['DB00455', 'DB00852'],
 nan,
 ['DB00869', 'DB00373'],
 ['DB00085'],
 nan,
 ['DB00316', 'DB00647'],
 ['DB00316', 'DB00647'],
 nan,
 ['DB00999', 'DB00177'],
 ['DB13913', 'DB01174'],
 nan,
 ['DB00405', 'DB00852'],
 ['DB00852'],
 ['DB01114', 'DB00388'],
 ['DB00673'],
 ['DB00281', 'DB00750'],
 ['DB00977', 'DB00304'],
 ['DB00443', 'DB02300'],
 nan,
 nan,
 ['DB00783'],
 ['DB01744', 'DB00825'],
 ['DB00783'],
 ['DB00316', ['DB00316'], 'DB00201'],
 ['DB00316', ['DB00316'], 'DB00201', 'DB00318'],
 ['DB11365'],
 [['DB00977']],
 nan,
 [['DB00977']],
 nan,
 ['DB09287'],
 nan,
 ['DB09287'],
 nan,
 nan,
 [[nan]],
 nan,
 ['DB14740'],
 ['DB00956', 'DB01050'],
 nan,
 ['DB09146'],
 nan,
 ['DB00316', 'DB00201', 'DB01397'],

In [1258]:
a['loratadine']

'DB00455'

In [1267]:
# ethinylestradiol
'DB00977'
# levonorgestrel
'DB00367'
# norgestrel
DB09389
# desogestrel
DB00304
# norethindrone/ norethisterone
DB00717
# ferous fumarate
DB14491
# drospirenone
DB01395

# acetaminophen
'DB00316'
# hydrocodone
DB00956
# pseudoephedrine
DB00852
# caffeine
DB00201




In [1277]:
# z = ids[ids.db_id.isnull()].index
a['wera'] = ['DB00977', 'DB00717']

In [1282]:
tmp2 = ids[ids.db_id.isnull()]
tmp2

,brand,db_id,actives
384,blistex,NaN,Generic Name: topical emollients (TOP i kal ee...
385,blistex lip balm,NaN,Generic Name: topical emollients (TOP i kal ee...
532,cepacol dual relief sore throat,NaN,NaN
686,coricidin hbp cold & flu,NaN,NaN
713,cryselle,NaN,NaN
801,derma-smoothe,NaN,NaN
925,drixoral cold,NaN,NaN
1030,epiceram,NaN,NaN
1033,epiduo forte,NaN,NaN
1206,flu,NaN,NaN


In [1292]:
tmp2.db_id = tmp2.brand.map(a)

def remove_nan(x):
    try:
        if len(x) > 1:
            return x
        else:
            if str(x[0]) != 'nan':
                return x
            else:
                return np.nan
    except:
        return x
tmp2.db_id = tmp2.db_id.apply(remove_nan)
tmp2

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,brand,db_id,actives
384,blistex,NaN,Generic Name: topical emollients (TOP i kal ee...
385,blistex lip balm,NaN,Generic Name: topical emollients (TOP i kal ee...
532,cepacol dual relief sore throat,NaN,NaN
686,coricidin hbp cold & flu,NaN,NaN
713,cryselle,NaN,NaN
801,derma-smoothe,NaN,NaN
925,drixoral cold,NaN,NaN
1030,epiceram,NaN,NaN
1033,epiduo forte,NaN,NaN
1206,flu,NaN,NaN


In [1295]:
# tmp2.drop(columns = 'actives', inplace = True)
tmp2.dropna(inplace = True)
tmp2

<ipython-input-1295-5c2756814ced>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp2.dropna(inplace = True)


,brand,db_id
1662,levora,"[DB00977, DB00367]"
1709,lo loestrin fe,"[DB00977, DB00717, DB14491]"
1716,loestrin fe 1.5 / 30,"[DB00977, DB00717]"
1730,loratidine,DB00455
1735,lorcet 10 / 650,"[DB00316, DB00956]"
1953,monistat 1-day or night combination pack,DB01110
1954,monistat 3-day combination pack,DB01110
1956,monistat 7-day combination pack,DB01110
2282,panlor dc,"[DB00316, DB01551, DB00201]"
2390,pirmella 1 / 35,"[DB00977, DB00717]"


In [1298]:
ids.loc[tmp2.index, 'db_id'] = tmp2.db_id
ids.loc[tmp2.index]

,brand,db_id,actives
1662,levora,"[DB00977, DB00367]",Generic Name: levonorgestrel and ethinyl estra...
1709,lo loestrin fe,"[DB00977, DB00717, DB14491]",Generic Name: norethindrone acetate and ethiny...
1716,loestrin fe 1.5 / 30,"[DB00977, DB00717]",NaN
1730,loratidine,DB00455,NaN
1735,lorcet 10 / 650,"[DB00316, DB00956]",NaN
1953,monistat 1-day or night combination pack,DB01110,NaN
1954,monistat 3-day combination pack,DB01110,NaN
1956,monistat 7-day combination pack,DB01110,NaN
2282,panlor dc,"[DB00316, DB01551, DB00201]",NaN
2390,pirmella 1 / 35,"[DB00977, DB00717]",NaN


In [1306]:
# ids.drop(columns = 'actives', inplace = True)
ids.dropna(inplace = True)
ids

,brand,db_id
0,abacavir,DB01048
1,abatacept,DB01281
2,abilify,DB01238
3,abilify maintena,DB01238
4,abiraterone,DB05812
...,...,...
3450,zyrtec-d,DB00788
3451,zyrtec-d 12 hour,DB13867
3452,zytiga,DB05812
3453,zyvox,DB00601


In [1307]:
# ids.to_csv('ids.csv', index = False, header = True)

In [1313]:
ids = {i[0]:i[1] for i in zip(ids.brand, ids.db_id)}
ids

{'abacavir': 'DB01048',
 'abatacept': 'DB01281',
 'abilify': 'DB01238',
 'abilify maintena': 'DB01238',
 'abiraterone': 'DB05812',
 'abobotulinumtoxina': 'DB00083',
 'abraxane': 'DB01229',
 'abreva': 'DB00632',
 'absorbine jr.': 'DB00825',
 'absorica': 'DB00982',
 'abstral': 'DB00813',
 'acamprosate': 'DB00659',
 'acarbose': 'DB00284',
 'accolate': 'DB00549',
 'accupril': 'DB00881',
 'accutane': 'DB00982',
 'acebutolol': 'DB01193',
 'acetaminophen': 'DB00316',
 'acetazolamide': 'DB00819',
 'acetic acid': 'DB03166',
 'acetohydroxamic acid': 'DB00551',
 'acetylcysteine': 'DB06151',
 'acidophilus': 'DB15823',
 'aciphex': 'DB01129',
 'acitretin': 'DB00459',
 'aclidinium': 'DB08897',
 'acrivastine': 'DB09488',
 'actemra': 'DB06273',
 'acthar': 'DB01285',
 'acticlate': 'DB00254',
 'actiq': 'DB00813',
 'actonel': 'DB00884',
 'actos': 'DB01132',
 'acular ls': 'DB00465',
 'acuvail': 'DB00465',
 'acyclovir': 'DB00787',
 'aczone': 'DB00250',
 'adalat cc': 'DB01115',
 'adalimumab': 'DB00051',
 'ad

In [1354]:
rv_exploded.db_id = rv_exploded.drug.map(ids)
# rv_exploded.drop(columns = 'index', inplace = True)
# rv_exploded.reset_index(drop = True)
# rv_exploded = rv_exploded.apply(lambda x: x.explode())
# rv_exploded.drop(columns = 'level_0', inplace = True)
# rv_exploded = rv_exploded.explode('db_id')
rv_exploded

,drug,condition,review,rating,useful_count,db_id
0,resorcinol,acne,"""have loved this product since i began using i...",10.0,3,DB11085
1,sulfur,acne,"""have loved this product since i began using i...",10.0,3,NaN
2,diphenhydramine,pruritus,"""hc derma-pax was my &quot;go to&quot; anti-it...",10.0,0,DB01075
3,hydrocortisone,pruritus,"""hc derma-pax was my &quot;go to&quot; anti-it...",10.0,0,DB00741
4,diphenhydramine,pruritus,"""i am an avid hiker and for bug bites, poison ...",10.0,4,DB01075
...,...,...,...,...,...,...
246754,loryna,birth control,"""weird - was on yaz (brand name) for over a ye...",2.0,5,"[DB01395, DB00977]"
246755,loryna,acne,"""i had tried just about everything for acne fr...",7.0,6,"[DB01395, DB00977]"
246755,loryna,acne,"""i had tried just about everything for acne fr...",7.0,6,"[DB01395, DB00977]"
246755,loryna,acne,"""i had tried just about everything for acne fr...",7.0,6,"[DB01395, DB00977]"


In [1368]:
rv_exploded.loc[246755].db_id.values

array([list(['DB01395', 'DB00977']), list(['DB01395', 'DB00977']),
       list(['DB01395', 'DB00977']), list(['DB01395', 'DB00977'])],
      dtype=object)

In [1370]:
x = list(rv_exploded[rv_exploded.drug == 'minastrin 24 fe'].index)
rv_exploded.loc[x,'db_id'] = list(['DB00977', 'DB00717', 'DB14491'])

ValueError: cannot set using a multi-index selection indexer with a different length than the value

In [1396]:
rv_exploded[(rv_exploded.drug == 'minastrin 24 fe')]
# rv_exploded.loc[z]

,drug,condition,review,rating,useful_count,db_id
58102,minastrin 24 fe,birth control,"""i first tried lo loestrin fe and it made me f...",9.0,1,DB00717
58103,minastrin 24 fe,birth control,"""i started this birth control in january 2016....",4.0,1,DB00717
58104,minastrin 24 fe,birth control,"""i did not like this drug at all, had to switc...",3.0,1,DB00717
58105,minastrin 24 fe,birth control,"""i love taking this pill. set an alarm (or two...",10.0,1,DB00717
58106,minastrin 24 fe,birth control,"""i&#039;ve been on this for about 3 months. i&...",3.0,0,DB00717
...,...,...,...,...,...,...
244210,minastrin 24 fe,birth control,"""was on for 4 months or so... it was ok. i did...",6.0,22,DB00977
244211,minastrin 24 fe,birth control,"""i&#039;ve never been on birth control until n...",2.0,35,DB00977
244212,minastrin 24 fe,birth control,"""i&#039;ve been on minastrin for 5 months now....",8.0,21,DB00977
244213,minastrin 24 fe,birth control,"""i was switched to minastrin from loestrin, ju...",8.0,15,DB00977


In [1392]:
rv_exploded.loc[z, 'db_id'] = 'DB00717'

In [1507]:
z = rv_exploded[(rv_exploded.drug.str.contains("flu")) & rv_exploded.db_id.isnull()].index
zz = rv_exploded.loc[z][~rv_exploded.loc[z].duplicated()].index
rv_exploded.loc[zz]

,drug,condition,review,rating,useful_count,db_id


In [1484]:
# rv_exploded.drop(index = 25016, inplace = True)

In [1500]:
rv_exploded.loc[zz, 'db_id'] = 'DB00977'

In [1501]:
q = rv_exploded[(rv_exploded.drug.str.contains("derma-pax"))].index
rv_exploded = rv_exploded.append(rv_exploded.loc[q], ignore_index = True)

In [1505]:
rv_exploded[(rv_exploded.drug == "flu")]

,drug,condition,review,rating,useful_count,db_id
24873,flu,allergic rhinitis,"""i have used drixoral for years and it is the ...",10.0,1,DB00405
259228,flu,allergic rhinitis,"""i have used drixoral for years and it is the ...",10.0,1,NaN


In [1506]:
rv_exploded.loc[259228, 'db_id'] = 'DB00852'

In [1404]:
a['cryselle'] = ['DB00977', 'DB09389']

In [1512]:
rv_exploded.isnull().sum()

drug               0
condition       1776
review             0
rating             0
useful_count       0
db_id              0
dtype: int64

In [1513]:
rv_exploded.shape

(285604, 6)

In [1582]:
x = rv_exploded[rv_exploded.condition.str.contains('</span') == True].index
rv_exploded.loc[x, 'condition'] = np.nan

In [1634]:
rv_exploded[rv_exploded.condition.isnull()]

,drug,condition,review,rating,useful_count,db_id
13086,bacitracin,NaN,"""i was allergic to this. just as bad as i was ...",1.0,2,DB00626
13087,polymyxin b,NaN,"""i was allergic to this. just as bad as i was ...",1.0,2,DB00781
13104,albuterol,NaN,"""the design of the new inhaler needs to be cha...",1.0,7,DB01001
13105,ipratropium,NaN,"""the design of the new inhaler needs to be cha...",1.0,7,DB00332
13200,naproxen,NaN,"""this medicine zapped the migraine away quickl...",8.0,0,DB00788
...,...,...,...,...,...,...
288721,remeron soltab,NaN,"""excellent medication, only thing to work afte...",9.0,29,DB00370
288849,milnacipran,NaN,"""i have been taking savella for about a week n...",8.0,3,DB04896
288937,milnacipran,NaN,"""in the past 10 yrs. i&#039;ve had l-4/5 back ...",3.0,7,DB04896
289091,hydromet,NaN,"""amazing for pain and cough alike""",9.0,5,DB00338


In [1611]:
z = rv_exploded[(rv_exploded.drug.str.contains('camrese')) & rv_exploded.condition.isnull()].index
rv_exploded.loc[z]

,drug,condition,review,rating,useful_count,db_id
55315,camrese,NaN,"""i&#039;m 46 and i have been having menopausal...",2.0,2,"[DB00977, DB00367]"
55316,camrese,NaN,"""i&#039;m 46 and i have been having menopausal...",2.0,2,"[DB00977, DB00367]"
55317,camrese,NaN,"""i&#039;m 46 and i have been having menopausal...",2.0,2,"[DB00977, DB00367]"
55318,camrese,NaN,"""i&#039;m 46 and i have been having menopausal...",2.0,2,"[DB00977, DB00367]"
259866,camrese,NaN,"""i&#039;m 46 and i have been having menopausal...",2.0,2,"[DB00977, DB00367]"
259867,camrese,NaN,"""i&#039;m 46 and i have been having menopausal...",2.0,2,"[DB00977, DB00367]"
259868,camrese,NaN,"""i&#039;m 46 and i have been having menopausal...",2.0,2,"[DB00977, DB00367]"
259869,camrese,NaN,"""i&#039;m 46 and i have been having menopausal...",2.0,2,"[DB00977, DB00367]"


In [1631]:
rv_exploded.isnull().sum()

drug               0
condition       2080
review             0
rating             0
useful_count       0
db_id              3
dtype: int64

In [1626]:
rv_exploded.reset_index(drop = True, inplace = True)
rv_exploded = rv_exploded.apply(lambda x: x.explode('db_id'))
rv_exploded = rv_exploded[~rv_exploded.duplicated()]

rv_exploded.drop(index = rv_exploded[rv_exploded.db_id.isnull()].index, inplace = True)

In [1638]:
rv_group = rv_exploded.groupby('db_id').agg(pd.Series.tolist)
rv_group.drug = rv_group.drug.apply(pd.unique)
rv_group.condition = rv_group.condition.apply(pd.unique)
rv_group.head()

,drug,condition,review,rating,useful_count
db_id,,,,,
DB00002,"[erbitux, cetuximab]","[colorectal cance, head and neck cance, squamo...","[""i have stage 4 colon cancer with liver mets....","[9.0, 6.0, 1.0, 7.0, 8.0, 9.0, 8.0, 7.0, 9.0, ...","[2, 2, 1, 8, 7, 13, 47, 43, 2, 2, 1, 8, 7, 13,..."
DB00003,"[dornase alfa, pulmozyme]",[cystic fibrosis],"[""my outcome/experience with pulmozyme was goo...","[10.0, 10.0]","[9, 9]"
DB00005,[etanercept],"[rheumatoid arthritis, psoriatic arthritis, pl...","[""took enbrel with methotrexate for four year...","[5.0, 5.0, 10.0, 1.0, 10.0, 5.0, 6.0, 9.0, 10....","[18, 5, 10, 14, 5, 40, 5, 5, 47, 11, 5, 30, 36..."
DB00006,"[bivalirudin, angiomax]",[percutaneous coronary intervention],"[""excellent one time treatment"", ""excellent on...","[10.0, 10.0]","[0, 0]"
DB00007,"[leuprolide, lupron depot-ped, lupron, eligard]","[uterine fibroids, prostate cance, endometrios...","[""i was so skeptical about these shots, scared...","[10.0, 10.0, 3.0, 7.0, 2.0, 6.0, 1.0, 4.0, 6.0...","[1, 3, 0, 0, 0, 2, 10, 14, 0, 8, 4, 12, 20, 5,..."


In [1640]:
# rv_group.to_csv('rv_grouped.csv')

In [549]:
rv_exploded['db_id'] = rv_exploded['drug'].map(brand_ids)
rv_exploded.head()

,drug,condition,review,rating,useful_count,db_id
868,resorcinol,acne,"""have loved this product since i began using i...",10.0,3,DB11085
868,sulfur,acne,"""have loved this product since i began using i...",10.0,3,NaN
992,diphenhydramine,pruritus,"""hc derma-pax was my &quot;go to&quot; anti-it...",10.0,0,DB01075
992,hydrocortisone,pruritus,"""hc derma-pax was my &quot;go to&quot; anti-it...",10.0,0,DB00741
993,diphenhydramine,pruritus,"""i am an avid hiker and for bug bites, poison ...",10.0,4,DB01075


In [893]:
not_found = list(rv_exploded[rv_exploded['db_id'].isnull()]['drug'].unique())
not_found

['sulfur',
 'sodium biphosphate',
 'insulin isophane',
 'polycosanol',
 'zenchent',
 'fluzone',
 'guarana',
 'prevnar 13',
 'epiceram',
 'capzasin-hp',
 'osteo bi-flex',
 'colace',
 'peg-3350 with electolytes',
 'atripla',
 'mephyton',
 'glyxambi',
 'jalyn',
 'menopur',
 'sinemet',
 'anusol-hc suppositories',
 'citranatal 90 dha',
 'lagesic',
 'ortho-cept',
 'ornex',
 'juleber',
 'selsun blue',
 'tenoretic 50',
 'mylanta',
 'enstilar',
 'yaz',
 "phillips' milk of magnesia",
 'aggrenox',
 'cervidil',
 'corzide 80 / 5',
 'zantryl',
 'embeda',
 'panoxyl 10% acne foaming wash',
 'bupap',
 'cyred',
 'avar ls cleanser',
 'rosula',
 'co-trimoxazole',
 'solarcaine first aid medicated spray',
 'nuwiq',
 'aquavite-e',
 'vitafol-ob+dha',
 'multivitamin with iron',
 'magnesium amino acids chelate',
 'necon 7 / 7 / 7',
 'neutrasal',
 'multivitamin, prenatal',
 'integra',
 'fulvicin u / f',
 'rytary',
 'zoster vaccine live',
 'treximet',
 'cerefolin nac',
 'nuedexta',
 'pylera',
 'hyomax sl',
 'sebu

In [919]:
'loryna' in not_found

False

In [897]:
rv_drug_to_scrape = pd.DataFrame({'brand': not_found})
rv_drug_to_scrape
# rv_drug_to_scrape.to_csv('rv_to_scrape.csv')

,brand
0,sulfur
1,sodium biphosphate
2,insulin isophane
3,polycosanol
4,zenchent
...,...
838,oasis tears
839,zetran
840,hydromet
841,amethia lo


In [918]:
rv_drug_to_scrape[rv_drug_to_scrape.brand == 'loryna']

,brand


In [931]:
rv_to_scrape = pd.read_csv('rv_to_scrape.csv')
rv_to_scrape.brand = rv_drug_to_scrape.brand
rv_to_scrape.head()

,brand,active,db_id
0,sulfur,NaN,NaN
1,sodium biphosphate,NaN,NaN
2,insulin isophane,insulin human,DB00030
3,polycosanol,NaN,NaN
4,zenchent,norethindrone,DB00717


In [932]:
len(rv_to_scrape.brand), rv_to_scrape.brand.nunique()

(1526, 843)

In [933]:
rv_to_scrape = rv_to_scrape.groupby('brand').agg(pd.Series.tolist)
rv_to_scrape

,active,db_id
brand,,
adderall,[caffeine],[DB00201]
advair diskus,[butalbital],[DB00241]
advair hfa,[immune globulin],[nan]
advil cold and sinus liqui-gels,[acetaminophen],[DB00316]
advil pm,[methadone],[DB00333]
...,...,...
zubsolv,[docusate],[DB11089]
zutripro,[bupropion],[DB01156]
zymine,[tropicamide],[DB00809]


In [952]:
rv_to_scrape.isnull().sum()

active    0
db_id     0
dtype: int64

In [953]:
dict_x = {i[0]: i[1] for i in zip(rv_to_scrape.index, rv_to_scrape.db_id)}
dict_x

{'adderall': ['DB00201'],
 'advair diskus': ['DB00241'],
 'advair hfa': [nan],
 'advil cold and sinus liqui-gels': ['DB00316'],
 'advil pm': ['DB00333'],
 'afrin 4 hour extra moisturizing': ['DB00328'],
 'afrin nodrip sinus': ['DB01229'],
 'aggrenox': ['DB00304'],
 'airduo respiclick': ['DB00316'],
 'alavert d-12 hour allergy and sinus': ['DB00852'],
 'alcortin a': ['DB00158'],
 'aldactazide': ['DB11181'],
 'alesse': ['DB00977'],
 'alka-seltzer cold and sinus': ['DB00126'],
 'alka-seltzer plus cold': ['DB00977'],
 'allegra odt': ['DB00316'],
 'allegra-d 12 hour': ['DB09020'],
 'allegra-d 24 hour': ['DB00717'],
 'aller-ease': ['DB00766'],
 'allergy dn pe': ['DB00332'],
 'alphanine sd': ['DB00514'],
 'alprazolam intensol': ['DB00367'],
 'altavera': [nan],
 'alyacen 1 / 35': ['DB08822'],
 'amethia': ['DB00366'],
 'amethia lo': ['DB01390'],
 'amethyst': ['DB00440'],
 'anacin': ['DB00304'],
 'analpram-hc': ['DB00717'],
 'anaprox-ds': ['DB00717'],
 'anexsia': ['DB00514'],
 'angeliq': ['DB009

In [954]:
rv_exploded.db_id = rv_exploded.drug.map(dict_x)
rv_exploded.tail(10)

,level_0,index,drug,condition,review,rating,useful_count,db_id
246746,63600,12244,loryna,birth control,"""my experience on loryna has not been a good o...",1.0,2,NaN
246747,63601,12245,loryna,birth control,"""so i started bc yesterday... no joke i&#039;m...",4.0,3,NaN
246748,63602,12246,loryna,menstrual disorders,"""age 15 before pill, now 21. pill was original...",9.0,1,NaN
246749,63603,12247,loryna,acne,"""i have suffered from acne for years. i&#039;v...",6.0,3,NaN
246750,63604,12248,loryna,acne,"""i started birth control because i was sexuall...",1.0,2,NaN
246751,63605,12249,loryna,menstrual disorders,"""i love loryna! i&#039;ve always had extremely...",9.0,1,NaN
246752,63606,12250,loryna,birth control,"""it completely worked for birth control. even ...",5.0,4,NaN
246753,63607,12251,loryna,birth control,"""does what its intended to do!""",10.0,7,NaN
246754,63608,12252,loryna,birth control,"""weird - was on yaz (brand name) for over a ye...",2.0,5,NaN
246755,63609,12253,loryna,acne,"""i had tried just about everything for acne fr...",7.0,6,NaN


In [1669]:
x = rv_exploded[rv_exploded['drug'].str.contains('sulfur') & rv_exploded.db_id.isnull()].index
rv_exploded.loc[x]

,drug,condition,review,rating,useful_count,db_id


In [892]:
rv_exploded.loc[x, 'db_id'] = 'DB01395'

In [888]:
rv_exploded = rv_exploded.append(rv_exploded.loc[x], ignore_index = True)

In [670]:
rv_exploded.shape

(242869, 8)

In [546]:
len(brands), len(db_ids)

(3455, 3455)

In [548]:
brand_ids = {i[0]: i[1] for i in zip(brands, db_ids)}
brand_ids

{'abacavir': 'DB01048',
 'abatacept': 'DB01281',
 'abilify': 'DB01238',
 'abilify maintena': 'DB01238',
 'abiraterone': 'DB05812',
 'abobotulinumtoxina': 'DB00083',
 'abraxane': 'DB01229',
 'abreva': 'DB00632',
 'absorbine jr.': 'DB00825',
 'absorica': 'DB00982',
 'abstral': 'DB00813',
 'acamprosate': 'DB00659',
 'acarbose': 'DB00284',
 'accolate': 'DB00549',
 'accupril': 'DB00881',
 'accutane': 'DB00982',
 'acebutolol': 'DB01193',
 'acetaminophen': 'DB00316',
 'acetazolamide': 'DB00819',
 'acetic acid': 'DB03166',
 'acetohydroxamic acid': 'DB00551',
 'acetylcysteine': 'DB06151',
 'acidophilus': 'DB15823',
 'aciphex': 'DB01129',
 'acitretin': 'DB00459',
 'aclidinium': 'DB08897',
 'acrivastine': 'DB09488',
 'actemra': 'DB06273',
 'acthar': 'DB01285',
 'acticlate': 'DB00254',
 'actiq': 'DB00813',
 'actonel': 'DB00884',
 'actos': 'DB01132',
 'acular ls': 'DB00465',
 'acuvail': 'DB00465',
 'acyclovir': 'DB00787',
 'aczone': 'DB00250',
 'adalat cc': 'DB01115',
 'adalimumab': 'DB00051',
 'ad

In [501]:
len(rv_exploded) - len(reviews) == len(combos) - 24879

True

In [503]:
# group by drug name
rv_grouped = rv_exploded.groupby('drug').agg(pd.Series.to_list)

# get only unique values for condition
rv_grouped['condition'] = rv_grouped['condition'].apply(pd.unique)

# calculate total useful counts
rv_grouped['total_count'] = rv_grouped['useful_count'].apply(lambda x: sum(x))

# # add drugbank id column from web scrape
# rv_grouped['db_id'] = db_ids

rv_grouped.head()

,condition,review,rating,useful_count,total_count
drug,,,,,
a + d cracked skin relief,[bacterial skin infection],"[""i have severe cracked skin on my hands. i&#...",[10.0],[6],6
a / b otic,[otitis media],"[""it numbs the pain. it makes my ear feel heav...","[10.0, 10.0]","[20, 18]",38
abacavir,[hiv infection],"[""august 8th 2017 tested positive.\r\nviral lo...","[10.0, 5.0, 10.0, 4.0, 9.0, 7.0, 5.0, 10.0, 9....","[3, 1, 3, 4, 11, 2, 4, 7, 6, 1, 1, 12, 3, 1, 1...",913
abatacept,"[rheumatoid arthritis, juvenile idiopathic art...","[""after the first week of using orencia, i fel...","[10.0, 10.0, 9.0, 9.0, 1.0, 10.0, 3.0, 9.0, 10...","[92, 88, 92, 144, 3, 9, 11, 16, 14, 2, 63, 0, ...",1151
abilify,"[depression, bipolar disorde, major depressive...","[""used along with prozac (fluoxetine) helped a...","[8.0, 7.0, 9.0, 1.0, 1.0, 1.0, 1.0, 3.0, 3.0, ...","[0, 0, 5, 0, 0, 0, 1, 2, 6, 0, 3, 1, 3, 1, 4, ...",23756


In [1650]:
# separate into rows with or without a total_count to calculate mean rating
# calculate total useful counts
rv_group['total_count'] = rv_group['useful_count'].apply(lambda x: sum(x))

rv_no_count = rv_group[rv_group['total_count'] == 0]
rv_no_count.head()

,drug,condition,review,rating,useful_count,total_count
db_id,,,,,,
DB00006,"[bivalirudin, angiomax]",[percutaneous coronary intervention],"[""excellent one time treatment"", ""excellent on...","[10.0, 10.0]","[0, 0]",0
DB00027,[gramicidin],"[conjunctivitis, bacterial]","[""i was prescribed this for pink eye. not sure...",[1.0],[0],0
DB00053,"[cerezyme, imiglucerase]",[gaucher disease],"[""i had very good results using cerezyme. i w...","[9.0, 9.0]","[0, 0]",0
DB00189,[ethchlorvynol],[insomnia],"[""excellent and sustained relief not found in ...",[10.0],[0],0
DB00200,[hydroxocobalamin],[vitamin b12 deficiency],"[""severe shortness of breath, stomach pain, bl...","[1.0, 1.0]","[0, 0]",0


In [1652]:
def mean_rating(x):
    return sum(x['rating']) / len(x['rating'])

rv_no_count['rating'] = rv_no_count.apply(mean_rating, axis = 1)
rv_no_count.head()

<ipython-input-1652-7358398ef3fc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_no_count['rating'] = rv_no_count.apply(mean_rating, axis = 1)


,drug,condition,review,rating,useful_count,total_count
db_id,,,,,,
DB00006,"[bivalirudin, angiomax]",[percutaneous coronary intervention],"[""excellent one time treatment"", ""excellent on...",10.0,"[0, 0]",0
DB00027,[gramicidin],"[conjunctivitis, bacterial]","[""i was prescribed this for pink eye. not sure...",1.0,[0],0
DB00053,"[cerezyme, imiglucerase]",[gaucher disease],"[""i had very good results using cerezyme. i w...",9.0,"[0, 0]",0
DB00189,[ethchlorvynol],[insomnia],"[""excellent and sustained relief not found in ...",10.0,[0],0
DB00200,[hydroxocobalamin],[vitamin b12 deficiency],"[""severe shortness of breath, stomach pain, bl...",1.0,"[0, 0]",0


In [1659]:
rv_counts = rv_group[rv_group['total_count'] != 0]
rv_counts.head()

,drug,condition,review,rating,useful_count,total_count
db_id,,,,,,
DB00002,"[erbitux, cetuximab]","[colorectal cance, head and neck cance, squamo...","[""i have stage 4 colon cancer with liver mets....","[9.0, 6.0, 1.0, 7.0, 8.0, 9.0, 8.0, 7.0, 9.0, ...","[2, 2, 1, 8, 7, 13, 47, 43, 2, 2, 1, 8, 7, 13,...",246
DB00003,"[dornase alfa, pulmozyme]",[cystic fibrosis],"[""my outcome/experience with pulmozyme was goo...","[10.0, 10.0]","[9, 9]",18
DB00005,[etanercept],"[rheumatoid arthritis, psoriatic arthritis, pl...","[""took enbrel with methotrexate for four year...","[5.0, 5.0, 10.0, 1.0, 10.0, 5.0, 6.0, 9.0, 10....","[18, 5, 10, 14, 5, 40, 5, 5, 47, 11, 5, 30, 36...",5286
DB00007,"[leuprolide, lupron depot-ped, lupron, eligard]","[uterine fibroids, prostate cance, endometrios...","[""i was so skeptical about these shots, scared...","[10.0, 10.0, 3.0, 7.0, 2.0, 6.0, 1.0, 4.0, 6.0...","[1, 3, 0, 0, 0, 2, 10, 14, 0, 8, 4, 12, 20, 5,...",9435
DB00008,"[pegasys, peginterferon alfa-2a]",[hepatitis c],"[""i was treated with this twice on it&#039;s o...","[3.0, 10.0, 1.0, 10.0, 9.0, 1.0, 8.0, 7.0, 5.0...","[0, 3, 2, 5, 12, 13, 15, 20, 19, 33, 0, 0, 3, ...",248


In [1660]:
rv_counts['rating'] = rv_counts.apply(mean_rating, axis = 1)
rv_counts.head()

<ipython-input-1660-0350a5478531>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_counts['rating'] = rv_counts.apply(mean_rating, axis = 1)


,drug,condition,review,rating,useful_count,total_count
db_id,,,,,,
DB00002,"[erbitux, cetuximab]","[colorectal cance, head and neck cance, squamo...","[""i have stage 4 colon cancer with liver mets....",6.875000,"[2, 2, 1, 8, 7, 13, 47, 43, 2, 2, 1, 8, 7, 13,...",246
DB00003,"[dornase alfa, pulmozyme]",[cystic fibrosis],"[""my outcome/experience with pulmozyme was goo...",10.000000,"[9, 9]",18
DB00005,[etanercept],"[rheumatoid arthritis, psoriatic arthritis, pl...","[""took enbrel with methotrexate for four year...",8.246305,"[18, 5, 10, 14, 5, 40, 5, 5, 47, 11, 5, 30, 36...",5286
DB00007,"[leuprolide, lupron depot-ped, lupron, eligard]","[uterine fibroids, prostate cance, endometrios...","[""i was so skeptical about these shots, scared...",6.179487,"[1, 3, 0, 0, 0, 2, 10, 14, 0, 8, 4, 12, 20, 5,...",9435
DB00008,"[pegasys, peginterferon alfa-2a]",[hepatitis c],"[""i was treated with this twice on it&#039;s o...",6.791667,"[0, 3, 2, 5, 12, 13, 15, 20, 19, 33, 0, 0, 3, ...",248


In [1656]:
# # create function to normalize counts
# def normalize_counts(x):
#     return [(i / x['total_count']) for i in x['useful_count']]

# rv_counts['useful_count'] = rv_counts.apply(normalize_counts, axis = 1)
# rv_counts.head()

<ipython-input-1656-7b518ad17037>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_counts['useful_count'] = rv_counts.apply(normalize_counts, axis = 1)


,drug,condition,review,rating,useful_count,total_count
db_id,,,,,,
DB00002,"[erbitux, cetuximab]","[colorectal cance, head and neck cance, squamo...","[""i have stage 4 colon cancer with liver mets....","[9.0, 6.0, 1.0, 7.0, 8.0, 9.0, 8.0, 7.0, 9.0, ...","[0.008130081300813009, 0.008130081300813009, 0...",246
DB00003,"[dornase alfa, pulmozyme]",[cystic fibrosis],"[""my outcome/experience with pulmozyme was goo...","[10.0, 10.0]","[0.5, 0.5]",18
DB00005,[etanercept],"[rheumatoid arthritis, psoriatic arthritis, pl...","[""took enbrel with methotrexate for four year...","[5.0, 5.0, 10.0, 1.0, 10.0, 5.0, 6.0, 9.0, 10....","[0.00340522133938706, 0.0009458948164964056, 0...",5286
DB00007,"[leuprolide, lupron depot-ped, lupron, eligard]","[uterine fibroids, prostate cance, endometrios...","[""i was so skeptical about these shots, scared...","[10.0, 10.0, 3.0, 7.0, 2.0, 6.0, 1.0, 4.0, 6.0...","[0.00010598834128245893, 0.0003179650238473768...",9435
DB00008,"[pegasys, peginterferon alfa-2a]",[hepatitis c],"[""i was treated with this twice on it&#039;s o...","[3.0, 10.0, 1.0, 10.0, 9.0, 1.0, 8.0, 7.0, 5.0...","[0.0, 0.012096774193548387, 0.0080645161290322...",248


In [1657]:
# def weighted_mean_rating(x):
#     scores = [i * j for i in x['rating'] for j in x['useful_count']]
#     return sum(scores) / len(scores)

# rv_counts['rating'] = rv_counts.apply(weighted_mean_rating, axis = 1)
# rv_counts.head()

<ipython-input-1657-b082581032ab>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_counts['rating'] = rv_counts.apply(weighted_mean_rating, axis = 1)


,drug,condition,review,rating,useful_count,total_count
db_id,,,,,,
DB00002,"[erbitux, cetuximab]","[colorectal cance, head and neck cance, squamo...","[""i have stage 4 colon cancer with liver mets....",0.429688,"[0.008130081300813009, 0.008130081300813009, 0...",246
DB00003,"[dornase alfa, pulmozyme]",[cystic fibrosis],"[""my outcome/experience with pulmozyme was goo...",5.000000,"[0.5, 0.5]",18
DB00005,[etanercept],"[rheumatoid arthritis, psoriatic arthritis, pl...","[""took enbrel with methotrexate for four year...",0.040622,"[0.00340522133938706, 0.0009458948164964056, 0...",5286
DB00007,"[leuprolide, lupron depot-ped, lupron, eligard]","[uterine fibroids, prostate cance, endometrios...","[""i was so skeptical about these shots, scared...",0.022635,"[0.00010598834128245893, 0.0003179650238473768...",9435
DB00008,"[pegasys, peginterferon alfa-2a]",[hepatitis c],"[""i was treated with this twice on it&#039;s o...",0.282986,"[0.0, 0.012096774193548387, 0.0080645161290322...",248


In [1661]:
rv_counts.rating.describe()

count    1136.000000
mean        7.429048
std         1.690284
min         1.000000
25%         6.413211
50%         7.538462
75%         8.601786
max        10.000000
Name: rating, dtype: float64

In [1662]:
rv_no_count.rating.describe()

count    47.000000
mean      6.329787
std       3.663452
min       1.000000
25%       2.500000
50%       8.000000
75%      10.000000
max      10.000000
Name: rating, dtype: float64

In [1663]:
rv_new = pd.concat([rv_counts, rv_no_count], axis = 0)

# drop usefulcount and totalcount
rv_new.drop(columns = ['useful_count', 'total_count'], inplace = True)

rv_new.head()

,drug,condition,review,rating
db_id,,,,
DB00002,"[erbitux, cetuximab]","[colorectal cance, head and neck cance, squamo...","[""i have stage 4 colon cancer with liver mets....",6.875000
DB00003,"[dornase alfa, pulmozyme]",[cystic fibrosis],"[""my outcome/experience with pulmozyme was goo...",10.000000
DB00005,[etanercept],"[rheumatoid arthritis, psoriatic arthritis, pl...","[""took enbrel with methotrexate for four year...",8.246305
DB00007,"[leuprolide, lupron depot-ped, lupron, eligard]","[uterine fibroids, prostate cance, endometrios...","[""i was so skeptical about these shots, scared...",6.179487
DB00008,"[pegasys, peginterferon alfa-2a]",[hepatitis c],"[""i was treated with this twice on it&#039;s o...",6.791667


In [1664]:
len(rv_new)

1183

In [1703]:
sulfur = combos[combos.drug == 'sulfur']
sulfur = sulfur.groupby('drug').agg(pd.Series.tolist)
sulfur.condition = sulfur.condition.apply(pd.unique)
sulfur.rating = sulfur.apply(mean_rating, axis = 1)
sulfur.drop(columns = 'useful_count', inplace = True)
sulfur.reset_index(inplace = True)
sulfur['db_id'] = 'DB11104'
sulfur.set_index('db_id', inplace = True)
sulfur

,drug,condition,review,rating
db_id,,,,
DB11104,sulfur,"[acne, psoriasis, dandruff, rosacea, seborrhei...","[""have loved this product since i began using ...",8.541667


In [1736]:
rv_new = rv_new.append(xx)

In [1739]:
# rv_new.to_csv('reviews_final.csv')

In [1738]:
rv_new.drop(index = 'DB05382', inplace = True)

In [1728]:
x1 = rv_new.loc['DB05382']
x1 = x1.apply(pd.Series).T.fillna(method = 'ffill')

In [1729]:
x2 = rv_new.loc['DB00366']
x2 = x2.apply(pd.Series).T.fillna(method = 'ffill')

In [1735]:
xx = pd.concat([x1, x2])
xx['db_id'] = 'DB00366'
xx = xx.groupby('db_id').agg(pd.Series.tolist)
xx.drug = xx.drug.apply(pd.unique)
xx.rating = xx.apply(mean_rating, axis = 1)
xx

,drug,condition,review,rating
db_id,,,,
DB00366,"[diclegis, doxylamine, unisom sleeptabs, ameth...","[nausea/vomiting of pregnancy, nausea/vomiting...","[""my doctor recommended this for my morning si...",6.628834


In [1718]:
rv_new.loc['DB00165']

drug         [pyridoxine, supartz, vitamin b6, lutera, panl...
condition    [nausea/vomiting of pregnancy, osteoarthritis,...
review       ["my doctor recommended this for my morning si...
rating                                                 6.47654
Name: DB00165, dtype: object

# Not needed:

In [442]:
indices = rv_grouped.index.tolist()
indices[indices.index('Acnex')] = 'salicylic acid'
rv_grouped.index = indices
rv_grouped

,condition,review,rating,usefulCount,total_count
30,"[Diabetes, Type 1]","[""Great medicine. No side effects.""]",[9.0],[10],10
lidocaine,[Bacterial Skin Infection],"[""I have severe cracked skin on my hands. I&#...",[10.0],[6],6
A / B Otic,[Otitis Media],"[""It numbs the pain. It makes my ear feel heav...","[10.0, 10.0]","[20, 18]",38
Abacavir,[HIV Infection],"[""August 8th 2017 tested positive.\r\nViral lo...","[10.0, 5.0, 10.0, 4.0, 9.0, 7.0, 5.0, 10.0, 9....","[3, 1, 3, 4, 11, 2, 4, 7, 6, 1, 1, 12, 3, 1, 1...",913
Abatacept,"[Rheumatoid Arthritis, Juvenile Idiopathic Art...","[""After the first week of using Orencia, I fel...","[10.0, 10.0, 9.0, 9.0, 1.0, 10.0, 3.0, 9.0, 10...","[92, 88, 92, 144, 3, 9, 11, 16, 14, 2, 63, 0, ...",1151
...,...,...,...,...,...
von willebrand factor,[von Willebrand's Disease],"[""worked perfect for 20 years"", ""It was given ...","[10.0, 10.0]","[0, 5]",5
zidovudine,"[Occupational Exposure, HIV Infection]","[""I was stuck with a patient&#039;s bloody nee...","[2.0, 10.0, 10.0, 9.0]","[1, 2, 6, 1]",10
zinc glycinate,[Osteoarthritis],"[""I have fibromyalgia, osteoarthritis, chronic...",[8.0],[11],11
zinc oxide,"[Diaper Rash, Hemorrhoids (pramoxine / zinc ox...","[""I used this for Itch,rash and soreness recta...","[4.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0...","[11, 7, 12, 17, 3, 0, 6, 6, 4]",66


In [ ]:
indices[indices.index('Acanya')] = ['Benzoyl peroxide', 'Clindamycin']
indices[indices.index('Abilify Discmelt')] = 'Apripiprazole'

In [441]:
rv_grouped.loc['Benzalkonium']

KeyError: 'Benzalkonium'

In [363]:
list(rv_new[rv_new.index.str.contains('Acanya')]['review'])

[['"Iv&#039;e struggled with acne for years now.Once I started using Acanya, Ziana, and Solodyn, I can see a huge difference with my face . I had really bad acne and you could clearly tell it was there. I tried everything you could possibly think of and so far these products are the only ones that have been helpful. I would recommend seeing a dermatoligist if you have bad acne, they will most likely give you these products.Just wanted to share my experience, hope you find help with it."',
  '"Honestly, this medicine has been life changing for me. Before I used it, my acne was awful, mostly on my forehead but all around the rest of my face, too. My dermatologist prescribed it to me and within a month of using it my acne completely cleared up. Now three years later, I still use it and my face remains acne free!"',
  '"I absolutely love this product. it does take some time to fully work but the end results are amazing . if your skin does dry out try applying a small amount of moisturizer!

In [ ]:
# create function to process single row
# .apply(function, axis = 1)

In [109]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215063 entries, 0 to 232291
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   drugName     215063 non-null  object 
 1   condition    213869 non-null  object 
 2   review       215063 non-null  object 
 3   rating       215063 non-null  float64
 4   date         215063 non-null  object 
 5   usefulCount  215063 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 21.5+ MB


In [108]:
pd.unique(reviews['usefulCount'])

array([   4,   11,    7,   12,    3,    6,    2,    5,    8,    0,   17,
         26,    1,   14,   25,   23,   15,   18,   13,    9,   16,   19,
         31,   10,   24,   33,   51,   56,   34,   27,   21,   35,   20,
         40,   55,   28,   92,   41,   22,   82,   47,   53,   43,   88,
         32,   37,   73,   36,   67,   96,   63,   50,  101,   61,   70,
         98,   71,   42,   29,   39,   38,   57,   52,   81,   75,   62,
         46,   79,   30,  106,   45,  105,  107,   58,  252,   60,  111,
         44,   85,   77,   48,   66,   59,  114,  132,   86,   65,   49,
         69,   68,   54,  113,  249,   64,  104,   72,   83,  134,   94,
        223,  102,  117,   74,  110,   76,   78,   87,   84,  129,  170,
        185,  200,  195,  161,  141,  108,  121,  194,  119,  124,  109,
        168,  207,   95,  142,  139,   97,  143,  123,   90,  103,  137,
        130,  116,  201,  279,  151,  112,  126,  147,  283,  125,  365,
         91,  128,   80,  118,  167,  164,  138,  1

In [87]:
from tqdm import tqdm
# conditions = []
# all_reviews = []
ratings = []

for drug in tqdm(reviews['drugName'].unique()):
#     a = []
#     b = []
    c = {}
    
    for i, row in reviews.iterrows():
        if row['drugName'] == drug:
#             if row['condition'] not in a:
#                 a.append(row['condition'])
#             else:
#                 pass
            
#             b.append(row['review'])
            c[row['rating']] = row['usefulCount']
        
        else:
            pass
        
#     conditions.append(a)
#     all_reviews.append(b)
    ratings.append(c)

 12%|█▏        | 429/3671 [5:09:04<38:55:41, 43.23s/it]  


KeyboardInterrupt: 

In [70]:
# create new dataframe with each drug as a row

rv = pd.DataFrame({'drug': drugs,
                   'conditions': conditions,
                   'all_reviews': all_reviews})
rv.head()

,drug,conditions,all_reviews
0,Medroxyprogesterone,"[Abnormal Uterine Bleeding, Amenorrhea, Birth ...","[""Been on the depo injection since January 201..."
1,Phenylephrine,"[Eye Redness, mulation) (phenylephrine), Nasal...","[""Best eye drop I ever tried. Provided soothin..."
2,Silodosin,[Benign Prostatic Hyperplasia],"[""Really not sure why my urologist gave me sam..."
3,Resorcinol / sulfur,[Acne],"[""Have loved this product since I began using ..."
4,Methylin ER,"[ADHD, Depression, Narcolepsy]","[""After 20 years of seeing psychiatrists, I wa..."
